## Emotion Recognition on Twitter using BERT Classifier

Install transformer and prepare GPU device

In [173]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

torch.cuda.set_per_process_memory_fraction(0.7, 0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


Load model

In [174]:
model_name = 'bert-base-uncased' 
bert_model = BertModel.from_pretrained(model_name)

Load files into dataframes

In [175]:
import polars as pl
from pathlib import Path

input_path = Path("./") # You should modify this to your data folder.
df = pl.read_ndjson(input_path/"tweets_DM.json")
df = df.unnest("_source").unnest("tweet")
split_df = pl.read_csv(input_path/"data_identification.csv")
y_train = pl.read_csv(input_path/"emotion.csv")

### Preprocessing functions
1. **text_preprocess:** Remove trailing white spaces and <LH> tags. 
2. **preprocessing_for_bert:** Use the tokenizer from transoformers library to preproceess the texts to BERT-compatible format.

In [182]:
import re
from tqdm import tqdm
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
MAX_LEN = 128

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in tqdm(data):
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            padding='max_length',         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,      # Return attention mask
            truncation=True
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    ids = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return ids, masks
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    # text = re.sub(r'(@.*?)[\s]', ' ', text)


    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    text = re.sub(r'<LH>', '', text).strip()

    return text
def preprocess_df(df):
    texts = df['text'].map_elements(text_preprocessing)
    text_inputs, text_masks = preprocessing_for_bert(texts)

    return text_inputs, text_masks

In [178]:
# Concatenate the hashtags to the beginning of the text.
df = df.with_columns((pl.col("hashtags").list.join(" ") + " " + pl.col("text")).alias("text"))
# Remove unnecessary columns.
df = df.select(
    "tweet_id",
    pl.col("text"),
).join(split_df, "tweet_id")

# one-hot encoding
class_names = y_train['emotion'].unique().to_list()
def one_hot_encode(name):
    return class_names.index(name)
y_train = y_train.with_columns(y_train['emotion'].map_elements(one_hot_encode).alias("emotion"))

# Split the dataset into training and testing dataset
train_df = df.filter(df["identification"] == "train").join(y_train, "tweet_id").drop("identification")
test_df = df.filter(df["identification"] == "test").drop("identification")

# Change the label datatype to tensor.
train_labels = torch.tensor(train_df["emotion"].to_numpy())
train_df = train_df.drop("emotion")

print(train_df.head)
print(test_df.head)

<bound method DataFrame.head of shape: (1_455_563, 3)
┌──────────┬───────────────────────────────────┬─────────┐
│ tweet_id ┆ text                              ┆ emotion │
│ ---      ┆ ---                               ┆ ---     │
│ str      ┆ str                               ┆ i64     │
╞══════════╪═══════════════════════════════════╪═════════╡
│ 0x3140b1 ┆  Why Chester? <LH> <LH>           ┆ 7       │
│ 0x368b73 ┆  @JaredLeto you are the fish tha… ┆ 2       │
│ 0x296183 ┆ pouroutyourspirit He is coming b… ┆ 6       │
│ 0x2bd6e1 ┆  Dei is really such a beautiful … ┆ 4       │
│ …        ┆ …                                 ┆ …       │
│ 0x300ea2 ┆ Michigan Cannabis @CherylShuman … ┆ 4       │
│ 0x360b99 ┆  Finally watching the new episod… ┆ 3       │
│ 0x22eecf ┆  Doctor: I'm afraid you're going… ┆ 4       │
│ 0x2fb282 ┆ qualitybeer What I wouldn't give… ┆ 6       │
└──────────┴───────────────────────────────────┴─────────┘>
<bound method DataFrame.head of shape: (411_972, 2)
┌───────

In [183]:
train_inputs, train_masks = preprocess_df(train_df)
test_inputs, test_masks = preprocess_df(test_df)

100%|██████████| 411972/411972 [02:02<00:00, 3357.73it/s]


Split the validation dataset by randomly selecting 20% from the training dataset.

In [184]:
import random
def train_val_split(train_data, train_label, val_ratio, random_state):
    val_samples = int(len(train_data) * val_ratio)
    indices = list(range(len(train_data)))
    random.seed(random_state)
    random.shuffle(indices)

    train_indices = indices[val_samples:]
    val_indices = indices[:val_samples]

    X_train = train_data[train_indices]
    X_val = train_data[val_indices]
    y_train = train_label[train_indices]
    y_val = train_label[val_indices]

    return X_train, X_val, y_train, y_val

In [185]:
train_masks, val_masks, _, _ = train_val_split(
    train_masks, train_labels, val_ratio=0.2, random_state=42
)
train_inputs, val_inputs, train_labels, val_labels = train_val_split(
    train_inputs, train_labels, val_ratio=0.2, random_state=42
)

Use the preprocessed dataframes and tensor to generate torch dataset

In [186]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 16

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# # Create the DataLoader for our test set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

# # Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

**BERT classifier:** Uses BERT as an encoder to obtain embeddings of the sentence. It then connects the output of BERT to linear layers to generate class predictions.

In [187]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup


# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 8

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out),
            # nn.Softmax(dim=1),
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

CPU times: total: 0 ns
Wall time: 0 ns


1. Implement focal loss, a modified version of cross entropy that can effectively handle class imbalance problems. 
2. Calculate the alpha value for the focal loss by determining the inverse class frequency for each class. Additionally, write the training loop.

In [188]:
import time
import torch.nn.functional as F

# determine alpha 
unique_classes, class_counts = np.unique(y_train['emotion'].to_numpy(), return_counts=True)
class_frequencies = class_counts / len(y_train)
alpha = 1 / class_frequencies
alpha /= np.sum(alpha)
gamma = 2

class CategoricalFocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(CategoricalFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        ce_loss = F.cross_entropy(logits, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        elif self.reduction == 'none':
            return focal_loss
        else:
            raise ValueError("Invalid reduction option. Use 'mean', 'sum', or 'none'.")

# Specify loss function
loss_fn = CategoricalFocalLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

Release unused resources

In [189]:
import gc
gc.collect()

8

In [190]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)

c:\Users\zijun0502\anaconda3\envs\dm2023\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train the model, it will take ~7 hours.

In [191]:
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   1.369410   |     -      |     -     |   3.48   
   1    |   40    |   1.271237   |     -      |     -     |   3.34   
   1    |   60    |   1.195674   |     -      |     -     |   3.30   
   1    |   80    |   1.179040   |     -      |     -     |   3.29   
   1    |   100   |   1.082243   |     -      |     -     |   3.22   
   1    |   120   |   1.137884   |     -      |     -     |   3.24   
   1    |   140   |   1.039324   |     -      |     -     |   3.22   
   1    |   160   |   1.040538   |     -      |     -     |   3.26   
   1    |   180   |   1.161812   |     -      |     -     |   3.23   
   1    |   200   |   1.053216   |     -      |     -     |   3.23   
   1    |   220   |   1.018849   |     -      |     -     |   3.24   
   1    |   240   |   0.997680   |     -      |     -     |   3.23   


1. **evaluate_performance:** Evaluation function that will print the accuracy on the validation dataset.

In [192]:
from sklearn.metrics import accuracy_score

def evaluate_performance(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    # Get accuracy over the test set
    y_pred = np.argmax(probs, axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [193]:
probs = bert_predict(bert_classifier, val_dataloader)
evaluate_performance(probs, val_labels)

Accuracy: 64.56%


Genereate results for test dataset and write output file.

In [196]:
probs = bert_predict(bert_classifier, test_dataloader)
y_pred = np.argmax(probs, axis=1)

In [197]:
emotion_column = [class_names[i] for i in y_pred]
result_df = pl.DataFrame({
    'id': test_df['tweet_id'],
    'emotion': emotion_column
})
result_pd_df = result_df.to_pandas()
result_pd_df.to_csv('result.csv', index=False)
